In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
categorical = pd.read_csv('files_for_lab/categorical.csv')
numerical = pd.read_csv('files_for_lab/numerical.csv')
target = pd.read_csv('files_for_lab/target.csv')

data = pd.concat([categorical,numerical,target],axis=1)

In [3]:
display(data.head(),print('data.shape:',data.shape))

data.shape: (95412, 339)


,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B,TARGET_D
0,IL,36,H,F,3,L,E,C,T,2,...,12.0,10.0,4,7.741935,95515,0,4,39,0,0.0
1,CA,14,H,M,3,L,G,A,S,1,...,25.0,25.0,18,15.666667,148535,0,2,1,0,0.0
2,NC,43,U,M,3,L,E,C,R,2,...,16.0,5.0,12,7.481481,15078,1,4,60,0,0.0
3,CA,44,U,F,3,L,E,C,R,2,...,11.0,10.0,9,6.812500,172556,1,4,41,0,0.0
4,FL,16,H,F,3,L,F,A,S,2,...,15.0,15.0,14,6.864865,7112,1,2,26,0,0.0


None

In [4]:
data.isna().sum().sum()

0

In [5]:
data[data['FIRSTDATE_MM'].isna()] 

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B,TARGET_D


In [6]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [7]:
from sklearn.utils import resample

category_0 = data[data['TARGET_B'] == 0]
category_1 = data[data['TARGET_B'] == 1]

category_1_oversampled = resample(category_1, 
                                  replace=True, # Wichtig an diesem punkt! downsampling False
                                  n_samples = len(category_0))

print('Not Donate', category_0.shape)
print('Do Donate oversampled', category_1_oversampled.shape)

Not Donate (90569, 339)
Do Donate oversampled (90569, 339)


In [8]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)
data_upsampled['TARGET_B'].value_counts()

1    90569
0    90569
Name: TARGET_B, dtype: int64

In [9]:
X = data_upsampled.drop(['TARGET_B','TARGET_D'], axis = 1)
y = data_upsampled['TARGET_B']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [11]:
X_train_num = X_train.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(object)

In [12]:
X_test_num = X_test.select_dtypes(include = np.number)
X_test_cat = X_test.select_dtypes(object)

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
#create transformer
transformer = StandardScaler().fit(X_train_num)
x_standardized = transformer.transform(X_train_num)

In [15]:
#X_train
print(x_standardized.shape)
X_train_num_scaled = pd.DataFrame(x_standardized, columns=X_train_num.columns)

(126796, 330)


In [16]:
#X_test
X_test_num_scaled = pd.DataFrame(transformer.transform(X_test_num), columns=X_train_num.columns)

In [17]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
#create encoder
encoder = OneHotEncoder(drop='first').fit(X_train_cat)

In [19]:
#columns names
cols_cat = encoder.get_feature_names_out(input_features=X_train_cat.columns)

In [20]:
#train
encoded_train = encoder.transform(X_train_cat).toarray()
encoded_train = pd.DataFrame(encoded_train,columns=cols_cat)

In [21]:
#test
encoded_test = encoder.transform(X_test_cat).toarray()
encoded_test = pd.DataFrame(encoded_test,columns=cols_cat)

In [22]:
train_scaled = pd.concat([encoded_train,X_train_num_scaled],axis=1)
test_scaled = pd.concat([encoded_test,X_test_num_scaled],axis=1)
df = pd.concat([test_scaled,train_scaled],axis=0)
pd.set_option('display.max_columns', None)
display(df.head(3),print('X.shape:',df.shape))

X.shape: (181138, 354)


,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.183886,0.627309,0.157195,0.526801,-0.031352,-1.319702,-0.306382,1.330457,-0.613008,0.962329,-1.493046,2.832821,-1.284017,0.595539,-1.036359,-0.058589,-0.018594,-1.885385,-2.262037,2.032417,-0.211264,0.553797,-0.035269,0.616339,0.970337,-0.499983,0.724801,-2.040631,-0.207292,-0.195631,-0.262129,0.838946,-0.432842,-0.643463,0.322317,-0.177783,-0.081881,-0.371147,-0.231271,1.582454,0.115598,-0.329009,2.046227,0.551942,1.120890,1.535540,0.82639,-0.057865,0.172423,-0.244939,-0.102168,-0.161921,0.144969,0.223258,0.208538,-0.019867,-0.127935,-0.005689,-0.176817,-0.944700,0.578933,1.077914,0.162831,-0.519639,-0.271330,1.602893,1.478496,-0.609978,-0.703678,-0.820929,-1.165432,0.444763,1.175837,0.329782,-0.520721,-0.864178,-0.655263,-1.011486,-0.356392,1.072546,0.869499,1.215029,0.577467,0.281473,-0.365010,-0.501623,0.146116,1.015007,1.013633,1.071360,0.790301,-0.533693,-0.733787,-0.645299,-0.644494,-0.300682,-0.27438,-0.155931,1.775866,1.694106,2.404541,2.199375,0.885340,-0.880214,0.704037,-0.820540,-0.048922,0.358465,1.037900,0.938353,0.527419,0.727991,-0.779435,-0.328897,-0.554685,-0.235699,-0.935190,-0.641592,-0.700507,-0.266695,0.346255,-0.381128,-0.421116,-0.335879,-0.270938,2.486598,2.166674,1.639459,1.249444,0.816072,1.756900,-0.649934,1.572080,1.013562,1.166164,0.743889,0.553755,-0.637719,-0.548144,-0.590243,-0.140123,-0.564961,-0.620450,-0.529241,1.822674,1.372905,0.951389,0.615724,0.818262,-1.244290,1.142292,1.338022,1.234397,1.505058,1.359094,0.434183,-1.122741,-0.719405,-1.695302,0.356264,0.698418,2.060677,3.468695,1.008321,-0.006392,-1.029929,-0.621528,-1.600935,-0.075835,0.755984,1.806942,2.860214,0.759397,-0.076386,-0.567291,0.180112,0.610569,-0.559708,-0.729522,0.777287,-0.535625,0.742071,-0.580673,-0.432804,-0.405539,-0.211877,-0.130

None

### Random Forest Classifier

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
clf.fit(train_scaled, y_train)
print(clf.score(train_scaled, y_train))
print(clf.score(test_scaled, y_test))

y_pred = clf.predict(test_scaled)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.615058834663554
0.6133009458613964


1    27415
0    26927
Name: TARGET_B, dtype: int64

array([[16894, 10033],
       [10981, 16434]], dtype=int64)

### Cross Validation Score

In [24]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
cross_val_scores = cross_val_score(clf, train_scaled, y_train, cv=10)
print(np.mean(cross_val_scores))
print(cross_val_scores)

0.6121644473847437
[0.6137224  0.60757098 0.60473186 0.61309148 0.6115142  0.61908517
 0.61495386 0.61597918 0.60777664 0.61321871]


In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [26]:
print('prediction:', precision_score(y_test,y_pred))
print('recall:', recall_score(y_test,y_pred))
print('f1:', f1_score(y_test,y_pred))

prediction: 0.6209241697207843
recall: 0.5994528542768557
f1: 0.6099996288185294


In [27]:
display(confusion_matrix(y_test, y_pred))

array([[16894, 10033],
       [10981, 16434]], dtype=int64)

In [28]:
# 16786 -> donate
# 10853 -> would donate but predict as not donate
# 10141 -> wouldn't donate but predict as donate
# 16562 -> wouldn't donate 
tp = 16786
fp = 10853
fn = 10141
tn = 16562

In [29]:
len(data[data['TARGET_D']>0])

4843

In [30]:
donate_mean_p_person = data[data['TARGET_D']!=0]['TARGET_D'].mean()
print(round(donate_mean_p_person,2))
# mean donation of one donating person 15.62$

15.62


# Lab | Final regression model in "Health Care for All" Case

In [32]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeRegressor
model1 = DecisionTreeRegressor()
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
from sklearn.neighbors import KNeighborsRegressor
model3 = KNeighborsRegressor()

import numpy as np

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

# data should really be scaled here
model_pipeline = [model1, model2, model3]
model_names = ['Decision Tree Regressor', 'Linear Regression', 'KNN']
scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, train_scaled, y_train, cv=10))
    scores[model_name] = mean_score
print(scores)

# We can use the result to choose the best performing model

{'Decision Tree Regressor': 0.8325305622446713, 'Linear Regression': 0.05809772395290701, 'KNN': nan}


In [33]:
list(zip(model_pipeline, model_names))

[(DecisionTreeRegressor(), 'Decision Tree Regressor'),
 (LinearRegression(), 'Linear Regression'),
 (KNeighborsRegressor(), 'KNN')]

In [34]:
val_scores = {}

for model, model_name in zip(model_pipeline,model_names):
    model.fit(train_scaled, y_train)
    val_scores[model_name] = model.score(test_scaled,y_test)
print(val_scores)

{'Decision Tree Regressor': 0.8505639307242436, 'Linear Regression': 0.06594773466548043, 'KNN': 0.7181326555384873}


In [35]:
pred=DecisionTreeRegressor().fit(train_scaled,y_train).predict(test_scaled)

In [36]:
display(len(pred),len(y_test))

54342

54342

In [37]:
from sklearn.metrics import r2_score

print('R2 Score:', r2_score(y_test, pred))

R2 Score: 0.8514472966509969


In [38]:
display(confusion_matrix(y_test, pred))

array([[24909,  2018],
       [    0, 27415]], dtype=int64)

In [43]:
# 24890 -> donate
# 0     -> would donate but predict as not donate
# 2037  -> wouldn't donate but predict as donate
# 27415 -> wouldn't donate 
tp = 16786
fp = 0
fn = 2037
tn = 27415

post_cost=0.68

In [44]:
x = round((tn * donate_mean_p_person),2)
print('Amount saved money if use model:',round((x),2),'$')

Amount saved money if use model: 428341.4 $


In [45]:
x = round((fp * donate_mean_p_person),2)
print('Amount lost money if use model:',round((x),2),'$')

Amount lost money if use model: 0.0 $


In [46]:
x = round(((fn+tn)*post_cost),2)
print('Money to send Post:',round((x),2),'$')

Money to send Post: 20027.36 $


In [47]:
x = round((fn*post_cost),2)
print('Money to send Post for people who not donate:',round((x),2),'$')

Money to send Post for people who not donate: 1385.16 $
